In [1]:
import os
import json
import shutil

import session_processing_helper_c5 as helper
import utils_c5 as utils

import pandas as pd

In [2]:
data_dir = '/Users/rebekahzhang/data/behavior_data'
period = '20240123-20240415'
exp = "exp2"
cohort = "cohort_5"
data_folder = os.path.join(data_dir, period, exp, cohort)
print(data_folder)

/Users/rebekahzhang/data/behavior_data/20240123-20240415/exp2/cohort_5


# Quality Control

## Check session folders have both meta and events

In [3]:
def check_session_files(data_folder):
    files_check = []
    for entry in os.scandir(data_folder):
        if entry.is_dir():
            dir = entry.name
            session_path = os.path.join(data_folder, dir)
            events_found = False
            meta_found = False
            events_empty = True
            meta_empty = True
            
            required_files = [f for f in os.scandir(session_path) if f.is_file() and not f.name.startswith('.')]
            
            for file in required_files:
                if file.name.startswith("events_"):
                    events_found = True
                    if file.stat().st_size > 0:
                        events_empty = False
                elif file.name.startswith("meta_"):
                    meta_found = True
                    if file.stat().st_size > 0:
                        meta_empty = False
            
            files_check.append({
                'dir': dir,
                'events': events_found,
                'meta': meta_found,
                'events_empty': events_empty if events_found else None,
                'meta_empty': meta_empty if meta_found else None
            })

    files_check_df = pd.DataFrame(files_check).sort_values("dir")
    missing_meta = files_check_df[files_check_df.meta==False]
    missing_events = files_check_df[files_check_df.events==False]
    empty_meta = files_check_df[(files_check_df.meta==True) & (files_check_df.meta_empty==True)]
    empty_events = files_check_df[(files_check_df.events==True) & (files_check_df.events_empty==True)]
    
    return missing_meta, missing_events, empty_meta, empty_events

In [4]:
missing_events, missing_meta, empty_meta, empty_events = check_session_files(data_folder)
if not (missing_meta.empty and missing_events.empty and empty_meta.empty and empty_events.empty):
    print("\nFile check results:")
    if not missing_meta.empty:
        print("\nSessions missing meta files:")
        display(missing_meta)
    if not missing_events.empty:
        print("\nSessions missing events files:")
        display(missing_events)
    if not empty_meta.empty:
        print("\nSessions with empty meta files:")
        display(empty_meta)
    if not empty_events.empty:
        print("\nSessions with empty events files:")
        display(empty_events)
else:
    print("\nAll sessions have non-empty meta and events files.")


All sessions have non-empty meta and events files.


In [5]:
def delete_folders(folder_list, data_folder):
  if folder_list:
    for folder in folder_list:
      full_path = os.path.join(data_folder, folder)
      if os.path.exists(full_path):
        shutil.rmtree(full_path)
        print(f"Deleted folder: {full_path}")
      else:
        print(f"Folder not found: {full_path}")
  else:
    print("no sessions to delete")

In [6]:
delete_folders(missing_meta.dir.tolist(), data_folder)

no sessions to delete


In [7]:
delete_folders(missing_events.dir.tolist(), data_folder)

no sessions to delete


In [8]:
delete_folders(empty_meta.dir.tolist(), data_folder)

no sessions to delete


In [9]:
delete_folders(empty_events.dir.tolist(), data_folder)

no sessions to delete


## Generate and save sessions log

generate session log using meta data from each session and add columns of basic info to each session

In [10]:
# def modify_total_trial(row):
#     ending_code = row['ending_code'].lower()
#     if ending_code == 'pygame' or ending_code == 'manual':
#         return row['total_trial'] - 1
#     elif ending_code == 'miss':
#         return row['total_trial'] - 5
#     else:
#         return row['total_trial']

In [11]:
def modify_sessions_all(sessions_all):
    sessions_all['dir'] = sessions_all['date']+ '_' + sessions_all['time'] + '_' + sessions_all['mouse']
    sessions_all = sessions_all.sort_values('dir')
    sessions_all[['exp', 'group']] = sessions_all['exp'].str.extract(r'exp(\d)_(short|long)')
    sessions_all['group'] = sessions_all['group'].map({'short': 's', 'long': 'l'})
    
    # sessions_all['total_trial'] = sessions_all.apply(modify_total_trial, axis=1)
    # sessions_all = sessions_all.drop(['forward_file'], axis=1)
    return sessions_all

In [12]:
def generate_sessions_all(data_folder):
    """Generates a DataFrame using session metadata from JSON files.
    Args:
        data_folder (str): Path to the directory containing JSON files.
    Returns:
        pd.DataFrame: DataFrame containing session metadata, sorted by 'dir' column.
    """

    data = []
    for root, _, files in os.walk(data_folder):
        for file in files:
            if file.startswith("meta_") and file.endswith(".json"):
                path = os.path.join(root, file)
                try:
                    with open(path) as f:
                        session_data = json.load(f)
                        data.append(session_data)
                except Exception as e:
                    print(f"Error processing file {file}: {e}")

    sessions_all = pd.DataFrame(data)
    sessions_all = modify_sessions_all(sessions_all)
    return sessions_all

In [13]:
def generate_sessions_training(sessions_all):
    sessions_training = sessions_all.loc[sessions_all.training == 'regular'].reset_index()
    sessions_training = sessions_training.groupby('mouse', group_keys=False).apply(helper.assign_session_numbers)
    return sessions_training

In [14]:
def generate_session_logs(data_folder, save_logs=True):
    sessions_all = generate_sessions_all(data_folder)
    sessions_training = generate_sessions_training(sessions_all)
    if save_logs:
        utils.save_as_csv(df=sessions_all, folder=data_folder, filename='sessions_all.csv')
        utils.save_as_csv(df=sessions_training, folder=data_folder, filename='sessions_training.csv')

    return sessions_all, sessions_training

### re-run after every quality control steps

In [15]:
sessions_all, sessions_training = generate_session_logs(data_folder)

sessions_training.tail()

,index,mouse,date,time,exp,training,rig,pump_ul_per_turn,total_trial,total_reward,avg_tw,dir,group,session
280,176,RZ034,2024-04-15,12-42-19,2,regular,rig2,0.0686,356,700,2.12,2024-04-15_12-42-19_RZ034,s,56
281,59,RZ036,2024-04-15,12-43-20,2,regular,rig3,0.0659,524,700,1.27,2024-04-15_12-43-20_RZ036,s,56
282,75,RZ037,2024-04-15,13-23-53,2,regular,rig2,0.0686,270,700,5.86,2024-04-15_13-23-53_RZ037,l,56
283,117,RZ038,2024-04-15,13-39-14,2,regular,rig3,0.0659,129,285,6.03,2024-04-15_13-39-14_RZ038,l,56
284,295,RZ039,2024-04-15,14-48-29,2,regular,rig2,0.0686,466,700,1.79,2024-04-15_14-48-29_RZ039,l,56


## Remove unwanted sessions
doesn't need to run when data folder is cleaned 
<br>
sessions_all needs to be regenerated after every cleaning step

### Remove test sessions

In [16]:
def remove_sessions(sessions_to_remove, data_folder):
    for _, session_info in sessions_to_remove.iterrows():
        shutil.rmtree(os.path.join(data_folder, session_info.dir))

In [17]:
sessions_test = sessions_all.loc[sessions_all.mouse=='test']
if len(sessions_test) > 0:
    remove_sessions(sessions_test, data_folder)
    print("test sessions removed")
else:
    print("no test sessions to be deleted!")

no test sessions to be deleted!


### Check for short sessions

In [18]:
short_threshold = 20

In [19]:
sessions_short = sessions_all[(sessions_all['total_trial'] < short_threshold) | sessions_all['total_trial'].isna()]
if len(sessions_short)>0:
    display(sessions_short)
else: 
    print('no short sessions to be checked!')

no short sessions to be checked!


remove short sessions if needed

In [20]:
remove_sessions(sessions_short, data_folder)

# Process Events

load session log

In [21]:
sessions_training = utils.load_data(os.path.join(data_folder, 'sessions_training.csv'))

In [22]:
# trim events and check if events properly ended
def process_events(session_info, events):
    events = events.loc[events['session_trial_num'].between(0, session_info['total_trial'])]
    return events

In [23]:
problematic_sessions = pd.DataFrame(columns=sessions_training.columns)

for _, session_info in sessions_training.iterrows():
    try:
        events_processed_path = utils.generate_events_processed_path(data_folder, session_info)
        if os.path.isfile(events_processed_path):
            continue
        events = pd.read_csv(utils.generate_events_path(data_folder, session_info), low_memory=False)
        events = process_events(session_info, events)
        events_processed = events.groupby('session_trial_num', group_keys=False).apply(helper.add_trial_time)
        events_processed.to_csv(events_processed_path)
    except:
        problematic_sessions = pd.concat([problematic_sessions, session_info.to_frame().T], ignore_index=True)

In [24]:
if len(problematic_sessions) > 0:
    display(problematic_sessions)
else:
    print("all sessions are perfect! woohoo!")

all sessions are perfect! woohoo!


# Data set curation 

In [25]:
# deprecated rounds dict cuz we no longer run more than 1 session per day
mouse_list = utils.generate_mouse_list(sessions_all)
# mouse_list = ['RZ047','RZ049','RZ050','RZ051','RZ052','RZ053','RZ054','RZ055','RZ056']
sessions_by_date = sessions_training.groupby('date')

### Deal with missing sessions
this is not the proper way to deal with this. should have it populated with mean and variation

In [26]:
no_missing_sessions = True
for date, data in sessions_by_date:
    for mouse in mouse_list:
        mouse_by_date = data.loc[data['mouse'] == mouse]
        if len(mouse_by_date) < 1:
            no_missing_sessions = False
            print(f"on {date}, {mouse} has missing sessions")
if no_missing_sessions:
    print("no missing sessions!")

no missing sessions!


duplicate if you are sussed out of having to redo this

In [27]:
# put it into utils when you clean this up.
def backup(source_dir):
    """create a copy for source_dir in the same path parallel to source_dir"""
    data_folder = os.path.dirname(source_dir)
    source_name = os.path.basename(source_dir)
    destination_dir = os.path.join(data_folder, f"{source_name}_copy")
    if not os.path.isdir(destination_dir):
        shutil.copytree(source_dir, destination_dir)
        print(f"{os.path.basename(source_dir)} backed up")
    else:
        print(f"{os.path.basename(destination_dir)} already exist")

In [28]:
backup(data_folder)

cohort_5_copy already exist


### Deal with multiple sessions

In [29]:
days_to_stitch = []
mice_to_stitch = []
for date, data in sessions_by_date:
    for mouse in mouse_list:
        mouse_by_date = data.loc[data['mouse'] == mouse]
        if len(mouse_by_date) > 1:
            days_to_stitch.append(date)
            mice_to_stitch.append(mouse)
            print(f"on {date}, {mouse} has {len(mouse_by_date)} sessions")
if not days_to_stitch:
    print("no sessions to stitch!")

no sessions to stitch!


In [30]:
def get_session_basics(session_df):
    num_trials = session_df.session_trial_num.max() 
    last_trial = session_df.loc[session_df['session_trial_num'] == num_trials]

    num_blocks = last_trial.loc[(last_trial['key'] == 'trial') & (last_trial['value'] == 1), 'block_num'].iloc[0] + 1
    total_reward = round(session_df.reward_size.sum(), 2)
    total_time = round((session_df.session_time.max() - session_df.session_time.min()), 2)
    session_basics = {'num_blocks': num_blocks,
                      'num_trials': num_trials + 1,
                      'rewards': total_reward,
                      'session_time': total_time}
    return session_basics  

def assign_session_numbers(group):
    group.sort_values(by=['mouse', 'dir', 'date'], inplace=True)
    group['session'] = list(range(len(group)))
    return group

# Stitch sessions from the same mouse on the same day
def stitch_sessions(session_1, session_2):
    session_1_basics = get_session_basics(session_1)
    time_offset = session_1_basics['session_time']
    block_offset = session_1_basics['num_blocks']
    trial_offset = session_1_basics['num_trials']
    
    session_2.session_time = session_2.session_time + time_offset
    session_2.block_num = session_2.block_num + block_offset
    session_2.session_trial_num= session_2.session_trial_num + trial_offset

    stitched_session = pd.concat([session_1, session_2])
    return stitched_session

In [31]:
# run it if session stitching is needed, nothing would happen otherwise
# has to run more than once if there are more than 2 sessions. fix it for the next round pls
if not days_to_stitch:
    print("no sessions to stitch!")
else:
    for d, m in zip(days_to_stitch, mice_to_stitch):
        day = sessions_by_date.get_group(d)
        sessions_to_stitch = day[day['mouse'] == m]

        session_1_dir = utils.generate_events_processed_path(data_folder, sessions_to_stitch.iloc[0])
        session_2_dir = utils.generate_events_processed_path(data_folder, sessions_to_stitch.iloc[1])

        if os.path.exists(session_1_dir) and os.path.exists(session_2_dir):
            session_1 = pd.read_csv(session_1_dir)
            session_2 = pd.read_csv(session_2_dir)
            stitched_session = stitch_sessions(session_1, session_2) 
            #should change to stitch events. stitch sessions should be deleted. to follow nomanclature, all session should be renamed to events!!

            stitched_session.to_csv(session_1_dir, index=False)
            shutil.rmtree(os.path.join(data_folder, sessions_to_stitch.iloc[1].dir))
            print(f"{d} {m} session 2 deleted")
        else:
            print("one of the sessions do not exist")

no sessions to stitch!


In [32]:
sessions_all, sessions_training = generate_session_logs(data_folder)

sessions_training.tail()

,index,mouse,date,time,exp,training,rig,pump_ul_per_turn,total_trial,total_reward,avg_tw,dir,group,session
280,176,RZ034,2024-04-15,12-42-19,2,regular,rig2,0.0686,356,700,2.12,2024-04-15_12-42-19_RZ034,s,56
281,59,RZ036,2024-04-15,12-43-20,2,regular,rig3,0.0659,524,700,1.27,2024-04-15_12-43-20_RZ036,s,56
282,75,RZ037,2024-04-15,13-23-53,2,regular,rig2,0.0686,270,700,5.86,2024-04-15_13-23-53_RZ037,l,56
283,117,RZ038,2024-04-15,13-39-14,2,regular,rig3,0.0659,129,285,6.03,2024-04-15_13-39-14_RZ038,l,56
284,295,RZ039,2024-04-15,14-48-29,2,regular,rig2,0.0686,466,700,1.79,2024-04-15_14-48-29_RZ039,l,56


### Correct sessions log

In [33]:
def correct_sessions_training(data_folder, save_log=True):
    _, sessions_training = generate_session_logs(data_folder, save_logs=False)
    session_info_list = []
    for _, session_info in sessions_training.iterrows():
        events_processed = pd.read_csv(utils.generate_events_processed_path(data_folder, session_info), low_memory=False)
        session_basics = helper.get_session_basics(events_processed)
        session_basics['dir'] = session_info['dir']
        session_info_list.append(session_basics)
    sessions_info = pd.DataFrame(session_info_list)
    corrected_sessions_training = pd.merge(sessions_training, sessions_info, on="dir")
    corrected_sessions_training = corrected_sessions_training.drop(columns=['index', 'total_reward', 'total_trial', 'total_reward'])
    corrected_sessions_training = corrected_sessions_training.groupby('mouse', group_keys=False).apply(helper.assign_session_numbers)
    if save_log:
        utils.save_as_csv(df=corrected_sessions_training, folder=data_folder, filename='sessions_training.csv')
    return corrected_sessions_training

In [34]:
sessions_training = correct_sessions_training(data_folder)

# Analyze trials

## Generate Trials

In [35]:
def get_trial_basics(trial):
    """gets the df of a trial, extracts 5 things, and outputs as a dictionary"""
    trial_start = trial.loc[(trial['key'] == 'trial') & (trial['value'] == 1)].iloc[0]
    trial_end = trial.loc[(trial['key'] == 'trial') & (trial['value'] == 0)].iloc[0]

    trial_basics = {'session_trial_num': trial_start['session_trial_num'],
                    'block_trial_num': trial_start['block_trial_num'],
                    'block_num': trial_start['block_num'],
                    'start_time': trial_start['session_time'],
                    'end_time': trial_end['session_time']}
    return trial_basics

In [36]:
def generate_trials(session_info, events):
    trial_info_list = []
    for t in range(int(session_info.num_trials)):
        trial = events.loc[events['session_trial_num'] == t]
        trial_basics = get_trial_basics(trial)
        trial_info_list.append(trial_basics)
    trials = pd.DataFrame(trial_info_list)
    return trials

In [37]:
# generate all trials based on events processed
problematic_sessions = pd.DataFrame(columns=sessions_training.columns)
for _, session_info in sessions_training.iterrows():
    try: 
        trials_path = utils.generate_trials_path(data_folder, session_info)
        # if os.path.isfile(trials_path):
        #     continue
        
        events_processed = pd.read_csv(utils.generate_events_processed_path(data_folder, session_info))
        trials = generate_trials(session_info, events_processed)

        trials.to_csv(trials_path)
    except:
        problematic_sessions = pd.concat([problematic_sessions, session_info.to_frame().T], ignore_index=True)

In [38]:
if len(problematic_sessions) > 0:
    display(problematic_sessions)
else:
    print("all sessions are perfect! woohoo!")

all sessions are perfect! woohoo!


## Analyze trials

In [39]:
for _, session_info in sessions_training.iterrows():
    try:
        trials_analyzed_path = utils.generate_trials_analyzed_path(data_folder, session_info)
        # if os.path.isfile(trials_analyzed_path):
        #     continue
        
        session_by_trial = utils.load_data(utils.generate_events_processed_path(data_folder, session_info)).groupby('session_trial_num')
        trials = utils.load_data(utils.generate_trials_path(data_folder, session_info))
        trials_data = helper.get_trial_data_df(session_by_trial)
        trials_analyzed = pd.merge(trials, trials_data, on='session_trial_num')
        trials_analyzed['group'] = session_info.group #assigning trial type manually
        trials_analyzed.to_csv(trials_analyzed_path)
    except:
        display(session_info)

In [40]:
sessions_training

,mouse,date,time,exp,training,rig,pump_ul_per_turn,avg_tw,dir,group,session,num_blocks,num_trials,rewards,session_time,proper_end
0,RZ034,2024-01-30,09-52-34,2,regular,rig2,0.0680,0.47,2024-01-30_09-52-34_RZ034,s,0,1.0,397.0,210.0,2406.21,True
1,RZ036,2024-01-30,09-55-23,2,regular,rig3,0.0609,0.39,2024-01-30_09-55-23_RZ036,s,0,1.0,664.0,425.0,3601.08,True
2,RZ037,2024-01-30,10-36-04,2,regular,rig2,0.0680,1.70,2024-01-30_10-36-04_RZ037,l,0,1.0,252.0,370.0,3602.47,True
3,RZ038,2024-01-30,10-59-43,2,regular,rig3,0.0609,0.75,2024-01-30_10-59-43_RZ038,l,0,1.0,239.0,170.0,3607.86,True
4,RZ039,2024-01-30,11-40-20,2,regular,rig2,0.0680,1.18,2024-01-30_11-40-20_RZ039,l,0,1.0,245.0,275.0,3610.04,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,RZ034,2024-04-15,12-42-19,2,regular,rig2,0.0686,2.12,2024-04-15_12-42-19_RZ034,s,56,1.0,357.0,700.0,2320.38,True
281,RZ036,2024-04-15,12-43-20,2,regular,rig3,0.0659,1.27,2024-04-15_12-43-20_RZ036,s,56,1.0,525.0,700.0,3166.65,True
282,RZ037,2024-04-15,13-23-53,2,regular,rig2,0.0686,5.86,2024-04-15_13-23-53_RZ037,l,56,1.0,271.0,700.0,4774.87,True
283,RZ038,2024-04-15,13-39-14,2,regular,rig3,0.0659,6.03,2024-04-15_13-39-14_RZ038,l,56,1.0,130.0,285.0,2711.29,True
